In [1]:
import numpy as np
import statsmodels
from causalimpact import CausalImpact
import pandas as pd
from scipy import stats
import matplotlib
import seaborn as sns
import lifelines


In [2]:
dfc = pd.DataFrame(pd.read_pickle('compliance.pkl'))

In [3]:
dfb = pd.DataFrame(pd.read_pickle('survey.pkl'))
tiers = pd.DataFrame(pd.read_csv('tiers.csv'))


In [4]:
#tiers = tiers.drop('Unnamed: 0',axis = 1)
tiers = tiers.dropna()

dfb = dfb.merge(tiers,right_on = 'Fitbit',left_on = 'id')

In [5]:
df = pd.DataFrame(pd.read_csv('compliance_data.csv'))

In [6]:
dfb.head()

,id,stepChallenge,gender_1,race_1,parentincome_1,Extraversion_1,Extraversion_2,Extraversion_6,Agreeableness_1,Agreeableness_2,...,Conscientiousness_6,Openness_1,Openness_2,Openness_6,Neuroticism_1,Neuroticism_2,Neuroticism_6,Unnamed: 0,Tier,Fitbit
0,3M4MLH,1.0,2.0,1.0,1.0,3.750,3.750,3.0,4.000000,3.555556,...,2.666667,2.6,3.000000,2.555556,3.375,3.25,3.125,0,Tier1,3M4MLH
1,3NVWH4,0.0,2.0,2.0,5.0,3.875,4.250,NaN,3.000000,2.666667,...,NaN,3.0,3.555556,NaN,3.750,4.00,NaN,2,Tier1,3NVWH4
2,3YFW8D,1.0,2.0,5.0,3.0,2.875,NaN,NaN,3.000000,NaN,...,NaN,3.2,NaN,NaN,3.125,NaN,NaN,6,Tier3,3YFW8D
3,4RNCLT,1.0,1.0,2.0,7.0,3.500,NaN,NaN,3.555556,NaN,...,NaN,3.3,NaN,NaN,3.875,NaN,NaN,8,Tier3,4RNCLT
4,3TPKGY,0.0,2.0,2.0,3.0,4.000,3.625,NaN,2.888889,3.333333,...,NaN,3.8,4.555555,NaN,3.250,3.25,NaN,10,Tier2,3TPKGY


In [7]:
s1Start = " 2015-08-27"
s1End = '2015-08-27'
s2Start = "2015-08-27"
s2End = '2019-04-22'

In [8]:
df = df[['participID','date','week','complyPercent']]

In [9]:
df["Spring"] = 0
df.loc[((df.date >= s1Start)&(df.date <= s1End )) , 'Spring'] = 1
df.loc[((df.date >= s1Start)) , 'Start'] = 1
df = df[df['Start'] == 1]

dfF = df[df['Spring'] == 1]
groupedF =dfF.groupby('participID')
grouped =df.groupby('participID')
df['week_after_intervention'] = ((pd.to_datetime(df['date']) - pd.to_datetime(s2Start)).dt.days / 7)

In [10]:
'''grouped = grouped['complyPercent'].mean()
participI  =  grouped.index
mean = grouped.values

dict = {'participID': participI, 'semester_compliance': mean}  
    
dfF = pd.DataFrame(dict)'''

"grouped = grouped['complyPercent'].mean()\nparticipI  =  grouped.index\nmean = grouped.values\n\ndict = {'participID': participI, 'semester_compliance': mean}  \n    \ndfF = pd.DataFrame(dict)"

In [11]:
#df = df.merge(dfF,right_on = 'participID',left_on = 'participID')

In [12]:
df = df.merge(dfb,right_on = 'id',left_on = 'participID')

In [13]:
df['left_study'] = 0
df.loc[(df.date > s2Start)& (df.date < s2End),'left_study'] = 1


In [14]:
df = df.groupby('participID').tail(1)


In [15]:
df.left_study.value_counts()

1    494
0    120
Name: left_study, dtype: int64

In [16]:
df = df[df.week_after_intervention > 0]

In [19]:
df3 = df
df3 = df3.groupby('participID').tail(1)
df3 = df3[['week','left_study']]
df3 = df3.dropna()
df2 =df
df2.Extraversion_2 = df2.Extraversion_2.fillna(df2.Extraversion_1)
df2.Agreeableness_2 = df2.Agreeableness_2.fillna(df2.Agreeableness_1)
df2.Openness_2 = df2.Openness_2.fillna(df2.Openness_1)
df2.Neuroticism_2 = df2.Neuroticism_2.fillna(df2.Neuroticism_1)
df2.Conscientiousness_2 = df2.Conscientiousness_2.fillna(df2.Conscientiousness_1)
df2 = df2.groupby('participID').tail(1)
df4 = df2[['week','left_study','Extraversion_2','Agreeableness_2','Conscientiousness_2','Openness_2','Neuroticism_2','gender_1','race_1','parentincome_1']]
df2 = df2[['week','Tier','week_after_intervention','left_study','Extraversion_2','Agreeableness_2','Conscientiousness_2','Openness_2','Neuroticism_2','gender_1','parentincome_1','race_1']]
df2 = df2.dropna()
df4 = df4.dropna()

In [20]:
from lifelines import CoxPHFitter

In [ ]:
bad = df2[df2['semester_compliance'] < 80]

In [21]:
df.head()

,participID,date,week,complyPercent,Spring,Start,week_after_intervention,id,stepChallenge,gender_1,...,Openness_1,Openness_2,Openness_6,Neuroticism_1,Neuroticism_2,Neuroticism_6,Unnamed: 0,Tier,Fitbit,left_study
650,39WN3Y,2017-12-17,93,4.0,0,1.0,120.428571,39WN3Y,1.0,2.0,...,2.9,2.900000,3.111111,3.625,3.625,3.75,1063,Tier3,39WN3Y,1
1076,3G4DZ7,2016-09-22,61,31.0,0,1.0,56.000000,3G4DZ7,0.0,2.0,...,3.5,3.500000,NaN,2.750,2.750,NaN,290,Tier3,3G4DZ7,1
1356,3K88BR,2016-04-29,40,7.0,0,1.0,35.142857,3K88BR,0.0,1.0,...,3.3,3.300000,NaN,3.125,3.125,NaN,1391,Tier3,3K88BR,1
2075,3KSZ8V,2017-07-23,103,96.0,0,1.0,99.428571,3KSZ8V,1.0,1.0,...,3.7,3.666667,NaN,2.375,2.125,NaN,539,Tier1,3KSZ8V,1
2457,3LYX22,2016-08-11,55,78.0,0,1.0,50.000000,3LYX22,0.0,1.0,...,4.2,4.555555,NaN,2.375,1.750,NaN,409,Tier1,3LYX22,1


In [22]:
dfT = df2[['week','week_after_intervention','left_study']]

In [23]:
df5 = df2[['Tier','week_after_intervention','left_study','Extraversion_2','Agreeableness_2','Conscientiousness_2','Openness_2','Neuroticism_2','gender_1','race_1']]

In [24]:


cph = CoxPHFitter()
cph.fit(df5, duration_col='week_after_intervention', event_col='left_study' , strata = ['Tier','gender_1','race_1'])

cph.print_summary()

<lifelines.CoxPHFitter: fitted with 584 total observations, 112 right-censored observations>
             duration col = 'week_after_intervention'
                event col = 'left_study'
                   strata = ['Tier', 'gender_1', 'race_1']
   number of observations = 584
number of events observed = 472
   partial log-likelihood = -1445.04
         time fit was run = 2019-12-06 16:10:21 UTC

---
                      coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
Extraversion_2        0.12       1.13       0.07            -0.01             0.26                 0.99                 1.29
Agreeableness_2      -0.08       0.92       0.10            -0.28             0.12                 0.76                 1.12
Conscientiousness_2  -0.38       0.68       0.10            -0.57            -0.20                 0.57                 0.82
Openness_2           -0.06       0.94       0.10            -0.25             0.13             

In [ ]:
df2.Conscientiousness_2.plot(kind='hist')

In [ ]:
cph.check_assumptions()